# Census rankings

In [381]:
import pandas as pd
import numpy as np
import json
import math

In [382]:
options = pd.read_csv('./csv/lists/places_2020.csv')

In [383]:
areas = []
for i in options["code"]:
    try:
        areas.append(json.load(open(f'/Users/theojolliffe/Documents/Census Data/census-data-main/json/place/{i}.json', 'rb')))
    except FileNotFoundError:
        print(i)
        pass

E05009289
E05009290
E05009293
E05009294
E05009295
E05009297
E05009298
E05009299
E05009300
E05009301
E05009303
E05009306
E05009307
E05009312
E05011090


In [478]:
wards = []
for i in areas:
    if i['type']=='wd':
        wards.append(i)
        
regions = []
for i in areas:
    if i['type']=='rgn':
        regions.append(i)
        
lads = []
for i in areas:
    if i['type']=='lad':
        lads.append(i)
        
countries = []
for i in areas:
    if (i['type']=='ew')|(i['type']=='ctry'):
        countries.append(i)

In [479]:
countries[0]

{'code': 'K04000001',
 'name': 'England and Wales',
 'type': 'ew',
 'area': 15101353.83,
 'count': 1,
 'parents': [],
 'bounds': [[-6.41894, 49.86464], [1.76891, 55.81167]],
 'data': {'population': {'perc': {'2001': {'female': 51.34, 'male': 48.66},
    '2011': {'female': 50.83, 'male': 49.17},
    'change': {'female': -0.51, 'male': 0.51}},
   'value': {'2001': {'all': 52041505.0,
     'female': 26715791.0,
     'male': 25325714.0},
    '2011': {'all': 56075912.0, 'female': 28502536.0, 'male': 27573376.0},
    'change': {'all': 7.75, 'female': 6.69, 'male': 8.88}}},
  'age10yr': {'perc': {'2001': {'0-9': 12.31,
     '10-19': 12.76,
     '20-29': 12.6,
     '30-39': 15.52,
     '40-49': 13.36,
     '50-59': 12.59,
     '60-69': 9.29,
     '70plus': 11.57},
    '2011': {'0-9': 11.83,
     '10-19': 12.12,
     '20-29': 13.63,
     '30-39': 13.23,
     '40-49': 14.62,
     '50-59': 12.1,
     '60-69': 10.79,
     '70plus': 11.68},
    'change': {'0-9': -0.48,
     '10-19': -0.64,
     '20

In [480]:
for thisRegion in regions:
    
    # Empty array will be populted with ranks for each variable
    ranks = []

    # Filter areas with same parent
    sister_regions = []
    region_code = thisRegion['parents'][0]['code']
    for i in regions:
        if  region_code == i['parents'][0]['code']:
            sister_regions.append(i)

    # Loop through the various data variables
    for a in thisRegion['data']:
        if a in ['population', 'density', 'agemed']:
            b = 'value'
        else:
            b = 'perc'
        # Create nested object with localised rank
        thisRegion['data'][a][b+"_rank_local"] = {}    
        for c in ['2011', 'change']:
            thisRegion['data'][a][b+"_rank_local"][c] = {}
            for d in thisRegion['data'][a][b][c]:
                vari = thisRegion['data'][a][b][c][d]

                # Create sorted list of values from sister areas
                group_values = []
                for region in sister_regions:
                    group_values.append(region['data'][a][b][c][d])
                    group_values = [x if (type(x) == float) | (type(x) == int) else np.nan for x in group_values]
                group_values.sort(reverse=True)

                # Find index of value of area of interest
                try:
                    varRank = group_values.index(vari) + 1
                except ValueError:
                    print(thisRegion['name'])
                    pass

                # Convert bottom half rankings into negative values
                if varRank>len(group_values)/2:
                    varRank = varRank-len(group_values)-1

                thisRegion['data'][a][b+"_rank_local"][c][d] = varRank

                # Append ranking data to original array
                ranks.append({
                    'label': a+'_'+b+'_'+c+'_'+d, 
                    'locRank': varRank, 
                    'value': vari})

    # Sort in rank order
    ranks = sorted(ranks, key=lambda x: x['locRank']**2)
    thisRegion["Priorities"] = ranks

In [481]:
for thisWard in wards:
    
    # Empty array will be populted with ranks for each variable
    ranks = []

    # Filter areas with same parent
    sister_wards = []
    ward_code = thisWard['parents'][0]['code']
    for i in wards:
        if  ward_code == i['parents'][0]['code']:
            sister_wards.append(i)

    # Loop through the various data variables
    for a in thisWard['data']:
        if a in ['population', 'density', 'agemed']:
            b = 'value'
        else:
            b = 'perc'
        # Create nested object with localised rank
        thisWard['data'][a][b+"_rank_local"] = {}    
        for c in ['2011', 'change']:
            thisWard['data'][a][b+"_rank_local"][c] = {}
            for d in thisWard['data'][a][b][c]:
                vari = thisWard['data'][a][b][c][d]

                # Create sorted list of values from sister areas
                group_values = []
                for ward in sister_wards:
                    group_values.append(ward['data'][a][b][c][d])
                    group_values = [x if (type(x) == float) | (type(x) == int) else np.nan for x in group_values]
                group_values.sort(reverse=True)

                # Find index of value of area of interest
                try:
                    varRank = group_values.index(vari) + 1
                except ValueError:
                    print(thisWard['name'])
                    pass

                # Convert bottom half rankings into negative values
                if varRank>len(group_values)/2:
                    varRank = varRank-len(group_values)-1

                thisWard['data'][a][b+"_rank_local"][c][d] = varRank

                # Append ranking data to original array
                ranks.append({
                    'label': a+'_'+b+'_'+c+'_'+d, 
                    'locRank': varRank, 
                    'value': vari})

    # Sort in rank order
    ranks = sorted(ranks, key=lambda x: x['locRank']**2)
    thisWard["Priorities"] = ranks

Hampton Vale
Hampton Vale
Hampton Vale
Hampton Vale
Kingsway
Kingsway
Kingsway
Kingsway
St Agnes
St Agnes
St Agnes
St Agnes
St Neots East
St Neots East
St Neots East
St Neots East
Cranbrook
Cranbrook
Cranbrook
Cranbrook
Bridge
Bridge
Bridge
Bridge


In [482]:
for thisLad in lads:
    
    # Empty array will be populted with ranks for each variable
    ranks = []

    # Filter areas with same parent
    sister_lads = []
    lad_code = thisLad['parents'][0]['code']
    for i in range(len(lads)):
        if  lad_code == lads[i]['parents'][0]['code']:
            sister_lads.append(lads[i])

    # Loop through the various data variables
    for a in thisLad['data']:
        if a in ['population', 'density', 'agemed']:
            b = 'value'
        else:
            b = 'perc'
        # Create nested object with localised rank
        thisLad['data'][a][b+"_rank_local"] = {}    
        for c in ['2011', 'change']:
            thisLad['data'][a][b+"_rank_local"][c] = {}
            for d in thisLad['data'][a][b][c]:
                vari = thisLad['data'][a][b][c][d]

                # Create sorted list of values from sister areas
                group_values = []
                for lad in sister_lads:
                    group_values.append(lad['data'][a][b][c][d])
                    group_values = [x if (type(x) == float) | (type(x) == int) else np.nan for x in group_values]
                group_values.sort(reverse=True)

                # Find index of value of area of interest
                varRank = group_values.index(vari) + 1

                # Convert bottom half rankings into negative values
                if varRank>len(group_values)/2:
                    varRank = varRank-len(group_values)-1
                    
                natRank = thisLad['data'][a][b+"_rank"][c][d]
                if natRank > 168:
                    natRank = natRank-336-1

                thisLad['data'][a][b+"_rank_local"][c][d] = varRank

                # Append ranking data to original array
                ranks.append({
                    'label': a+'_'+b+'_'+c+'_'+d, 
                    'locRank': varRank, 
                    'natRank': natRank, 
                    'value': vari})

    # Sort in rank order
    ranks = sorted(ranks, key=lambda x: (abs(x['locRank']), -abs(x['value'])))
    thisLad["Priorities"] = ranks


In [483]:
lads[33]['data']['population']['value_rank']

{'2001': {'all': 200, 'female': 210, 'male': 191},
 '2011': {'all': 211, 'female': 213, 'male': 206},
 'change': {'all': 275, 'female': 234, 'male': 304},
 'overtake': {'all': ['Erewash']},
 'top_bottom': {'all': {-1: {'Barrow-in-Furness': -4.02},
   2: {'Manchester': 28.08},
   -2: {'Sefton': -3.24},
   -3: {'South Tyneside': -3.05},
   3: {'Newham': 26.28},
   1: {'Tower Hamlets': 29.57}}}}

In [484]:
topics = [{1: 'population', 2: 'value', 3: 'all'}, 
          {1: 'health', 2: 'perc', 3: 'good'},
          {1: 'travel', 2: 'perc', 3: 'car_van'},
          {1: 'agemed', 2: 'value', 3: 'all'},
          {1: 'economic', 2: 'perc', 3: 'employee'},
          {1: 'health', 2: 'perc', 3: 'bad'},
          {1: 'ethnicity', 2: 'perc', 3: 'black'},
          {1: 'economic', 2: 'perc', 3: 'student'},
         ]

In [485]:
ladCodes = []
for lad in lads:
    ladCodes.append(lad['code'])


In [486]:
df = pd.DataFrame(columns=
                  ['lad', '2001',  '2011',  'change', 'natRank', 'localRank', 'topic', 'parent'])
for j in range(len(topics)):
    df1 = pd.DataFrame(index=ladCodes, columns=
                      ['lad', '2001',  '2011',  'change', 'natRank', 'localRank', 'topic', 'parent'])
    for i in range(len(lads)):
        df1['lad'].iloc[i] = lads[i]['name']
        df1['2001'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]]['2001'][topics[j][3]]
        df1['2011'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]]['2011'][topics[j][3]]    
        df1['change'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]]['change'][topics[j][3]]
        df1['natRank'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]+"_rank"]['change'][topics[j][3]]    
        df1['localRank'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]+"_rank_local"]['change'][topics[j][3]]    
        df1['topic'].iloc[i] = topics[j][1]+"_"+topics[j][3]
        df1['parent'].iloc[i] = lads[i]['parents'][0]['name']
    df = pd.concat([df,df1])
df['natRank'] = np.where(df['natRank'] > 168, df['natRank']-337,df['natRank'])

In [487]:
for topic in topics:
    for lad in lads:
        
#       Find the areas that this area has overtaken
        v2001 = lad['data'][topic[1]][topic[2]]['2001'][topic[3]]
        v2011 = lad['data'][topic[1]][topic[2]]['2011'][topic[3]]
        dfT = df[(df['topic']==topic[1]+"_"+topic[3])&(df['2001']>v2001)&(df['2011']<v2011)]
        obje = []
        for i in range(dfT.shape[0]):
            obje.append(dfT.iloc[i]['lad'])

        lad['data'][topic[1]][topic[2]+"_rank"]['overtake'] = {}
        lad['data'][topic[1]][topic[2]+"_rank"]['overtake'][topic[3]] = obje
        
#       Find the area immediatly above or below
        below = lad['data'][topic[1]][topic[2]+'_rank_local']['2011'][topic[3]]
        above = lad['data'][topic[1]][topic[2]+'_rank_local']['2011'][topic[3]]-2
        if below<0:
            below=below+1
            above=above+1
        reg = lad['parents'][0]['name']
        name_above = df[(df['parent']==reg)&(df['topic']==topic[1]+"_"+topic[3])].sort_values('2011', ascending=False).iloc[above]['lad']
        name_below = df[(df['parent']==reg)&(df['topic']==topic[1]+"_"+topic[3])].sort_values('2011', ascending=False).iloc[below]['lad']
        if above>0:
            area_above = {'name': name_above,
                         'value': df[(df['lad']==name_above)&(df['topic']==topic[1]+"_"+topic[3])]['2011'].iloc[0]}
        else:
            area_above = 'NaN'
        if below<len(df[(df['parent']==reg)&(df['topic']==topic[1]+"_"+topic[3])]):
            area_below = {'name': name_below,
                          'value': df[(df['lad']==name_below)&(df['topic']==topic[1]+"_"+topic[3])]['2011'].iloc[0]}
        else:
            area_below = 'NaN'
        
        if 'above_below' not in lad['data'][topic[1]][topic[2]+'_rank_local'].keys():
            lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'] = {}

        if topic[3] not in lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'].keys():
            lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'][topic[3]] = {}
        
        lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'][topic[3]]["above"] = area_above
        lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'][topic[3]]["below"] = area_below
        if lad['name']=='Manchester':
            print('*********')
            print('topic:', topic[1]+"_"+topic[3])
            print('area_below:', area_below)
            print('area_above:', area_above)
            print('below:', below)
            print('above:', above)
            
        # Add top and bottom three biggest movers for each subject to every area
        df_topic = df[(df['topic']==topic[1]+"_"+topic[3])&(abs(df['natRank'])<4)]
        ob = {}
        for index, row in df_topic.iterrows():
            ob[row['natRank']]= {row['lad']: row['change']}
        if 'top_bottom' not in lad['data'][topic[1]][topic[2]+'_rank'].keys():
            lad['data'][topic[1]][topic[2]+'_rank']['top_bottom'] = {}
        lad['data'][topic[1]][topic[2]+'_rank']['top_bottom'][topic[3]]=ob


*********
topic: population_all
area_below: {'name': 'Liverpool', 'value': 466415}
area_above: NaN
below: 1
above: -1
*********
topic: health_good
area_below: {'name': 'Bury', 'value': 80.39}
area_above: {'name': 'Lancaster', 'value': 80.51}
below: 13
above: 11
*********
topic: travel_car_van
area_below: {'name': 'Rossendale', 'value': 78.13}
area_above: NaN
below: 0
above: -2
*********
topic: agemed_all
area_below: {'name': 'South Lakeland', 'value': 44.0}
area_above: NaN
below: 0
above: -2
*********
topic: economic_employee
area_below: {'name': 'South Ribble', 'value': 58.59}
area_above: NaN
below: 0
above: -2
*********
topic: health_bad
area_below: {'name': 'Pendle', 'value': 6.96}
area_above: {'name': 'Wigan', 'value': 7.18}
below: 17
above: 15
*********
topic: ethnicity_black
area_below: {'name': 'Trafford', 'value': 2.89}
area_above: NaN
below: 1
above: -1
*********
topic: economic_student
area_below: {'name': 'Preston', 'value': 6.33}
area_above: NaN
below: 1
above: -1


In [488]:
breaksLad = [i*336/10 for i in range(10)]
breaksLad

[0.0, 33.6, 67.2, 100.8, 134.4, 168.0, 201.6, 235.2, 268.8, 302.4]

In [489]:
lads[217]['data']['population']['value_rank']['change']['all']

244

In [490]:
5*1/336

0.01488095238095238

In [491]:
print(lads[86]['name'])
    
lads[86]['Priorities']

Lewes


[{'label': 'travel_perc_change_bus',
  'locRank': 3,
  'natRank': 15,
  'value': 4.11},
 {'label': 'economic_perc_change_self-employed',
  'locRank': 4,
  'natRank': 30,
  'value': 2.07},
 {'label': 'ethnicity_perc_change_asian',
  'locRank': -4,
  'natRank': -91,
  'value': 0.56},
 {'label': 'ethnicity_perc_change_white',
  'locRank': 5,
  'natRank': 107,
  'value': -1.39},
 {'label': 'travel_perc_change_taxi',
  'locRank': -5,
  'natRank': -30,
  'value': -0.24},
 {'label': 'agemed_value_2011_all',
  'locRank': 6,
  'natRank': 25,
  'value': 42.0},
 {'label': 'travel_perc_2011_bus', 'locRank': 6, 'natRank': 79, 'value': 7.87},
 {'label': 'age10yr_perc_2011_70plus',
  'locRank': 7,
  'natRank': 21,
  'value': 16.59},
 {'label': 'age10yr_perc_2011_60-69',
  'locRank': 7,
  'natRank': 66,
  'value': 13.53},
 {'label': 'tenure_perc_change_rent_free',
  'locRank': -7,
  'natRank': -77,
  'value': -0.93},
 {'label': 'travel_perc_change_other',
  'locRank': 7,
  'natRank': 65,
  'value': 0.

In [492]:
for lad in lads:
    topicList = ["travel", "tenure", "density", "age10yr"] 
    subjectList = ["bus", "other", "female", "male"]
    priorities = []
    for rank in lad['Priorities']:
        if ((rank['label'].split("_")[2] == "change") & (rank['label'].split("_")[0] not in topicList) & (rank['label'].split("_")[3] not in subjectList)):
            print(rank)
            priorities.append(rank)
            topicList.append(rank['label'].split("_")[0])
    lad['Priorities'] = priorities


{'label': 'economic_perc_change_unemployed', 'locRank': 1, 'natRank': 4, 'value': 2.3}
{'label': 'health_perc_change_good', 'locRank': -2, 'natRank': -105, 'value': 11.79}
{'label': 'ethnicity_perc_change_mixed', 'locRank': -2, 'natRank': -10, 'value': 0.22}
{'label': 'population_value_change_all', 'locRank': -6, 'natRank': -77, 'value': 3.86}
{'label': 'agemed_value_change_all', 'locRank': -6, 'natRank': 113, 'value': 2.7}
{'label': 'health_perc_change_bad', 'locRank': 1, 'natRank': -79, 'value': -4.05}
{'label': 'ethnicity_perc_change_mixed', 'locRank': 1, 'natRank': 113, 'value': 0.78}
{'label': 'economic_perc_change_unemployed', 'locRank': 2, 'natRank': 55, 'value': 1.43}
{'label': 'population_value_change_all', 'locRank': -4, 'natRank': -48, 'value': 2.64}
{'label': 'agemed_value_change_all', 'locRank': 6, 'natRank': 100, 'value': 2.78}
{'label': 'ethnicity_perc_change_white', 'locRank': 1, 'natRank': 2, 'value': -0.39}
{'label': 'agemed_value_change_all', 'locRank': 2, 'natRank':

In [477]:
lads[247]['Priorities']

[]

In [455]:
priorities

[{'label': 'tenure_perc_change_rent_free',
  'locRank': -1,
  'natRank': -3,
  'value': -1.94},
 {'label': 'age10yr_perc_change_10-19',
  'locRank': -1,
  'natRank': -13,
  'value': -1.87},
 {'label': 'travel_perc_change_taxi',
  'locRank': 1,
  'natRank': 27,
  'value': 0.24},
 {'label': 'age10yr_perc_change_20-29',
  'locRank': 2,
  'natRank': 15,
  'value': 3.04},
 {'label': 'travel_perc_change_bus',
  'locRank': -3,
  'natRank': -56,
  'value': -1.25},
 {'label': 'age10yr_perc_change_50-59',
  'locRank': 3,
  'natRank': 58,
  'value': 0.05},
 {'label': 'age10yr_perc_change_60-69',
  'locRank': -4,
  'natRank': -110,
  'value': 1.47},
 {'label': 'age10yr_perc_change_30-39',
  'locRank': -5,
  'natRank': -129,
  'value': -3.15},
 {'label': 'travel_perc_change_home',
  'locRank': 7,
  'natRank': 113,
  'value': -3.98},
 {'label': 'travel_perc_change_train_metro',
  'locRank': 7,
  'natRank': -161,
  'value': 1.55},
 {'label': 'travel_perc_change_moto',
  'locRank': 7,
  'natRank': 58,

In [456]:
triples = []
for p in priorities:
    triples.append([lads[247]['name']+"_"+p['label'].split("_")[0]+"_"+p['label'].split("_")[3], p['label'].split("_")[1]+"_"+p['label'].split("_")[2], p['value']])
    triples.append([lads[247]['name']+"_"+p['label'].split("_")[0]+"_"+p['label'].split("_")[3], p['label'].split("_")[1]+"_"+p['label'].split("_")[2]+"_localRank", p['locRank']])
triples

[['Manchester_tenure_rent', 'perc_change', -1.94],
 ['Manchester_tenure_rent', 'perc_change_localRank', -1],
 ['Manchester_age10yr_10-19', 'perc_change', -1.87],
 ['Manchester_age10yr_10-19', 'perc_change_localRank', -1],
 ['Manchester_travel_taxi', 'perc_change', 0.24],
 ['Manchester_travel_taxi', 'perc_change_localRank', 1],
 ['Manchester_age10yr_20-29', 'perc_change', 3.04],
 ['Manchester_age10yr_20-29', 'perc_change_localRank', 2],
 ['Manchester_travel_bus', 'perc_change', -1.25],
 ['Manchester_travel_bus', 'perc_change_localRank', -3],
 ['Manchester_age10yr_50-59', 'perc_change', 0.05],
 ['Manchester_age10yr_50-59', 'perc_change_localRank', 3],
 ['Manchester_age10yr_60-69', 'perc_change', 1.47],
 ['Manchester_age10yr_60-69', 'perc_change_localRank', -4],
 ['Manchester_age10yr_30-39', 'perc_change', -3.15],
 ['Manchester_age10yr_30-39', 'perc_change_localRank', -5],
 ['Manchester_travel_home', 'perc_change', -3.98],
 ['Manchester_travel_home', 'perc_change_localRank', 7],
 ['Manche

In [457]:
geogTriples = []
for lad in lads:
    geogTriples.append([lad['name'], "in", lad['parents'][0]['name']])
for ward in wards:
    geogTriples.append([ward['parents'][0]['name'], "contains", ward['name']])
for lad in lads:
    list1 = lad['bounds'][0]+lad['bounds'][1]
    for lad2 in lads:
        list2 = lad2['bounds'][0]+lad2['bounds'][1]
        listDif = [abs(list1[i]-list2[i]) for i in [0,1,2,3]]
        listDif.sort()
        if (sum(listDif[:3]) < 0.3) & (lad!=lad2):
            geogTriples.append([lad['name'], "near_"+str(round(sum(listDif[:3]), 2)), lad2['name']])


In [458]:
[triple for triple in geogTriples if triple[0] == "Fareham"]

[['Fareham', 'in', 'South East'],
 ['Fareham', 'contains', 'Fareham East'],
 ['Fareham', 'contains', 'Fareham North'],
 ['Fareham', 'contains', 'Fareham North-West'],
 ['Fareham', 'contains', 'Fareham South'],
 ['Fareham', 'contains', 'Fareham West'],
 ['Fareham', 'contains', 'Hill Head'],
 ['Fareham', 'contains', 'Locks Heath'],
 ['Fareham', 'contains', 'Park Gate'],
 ['Fareham', 'contains', 'Portchester East'],
 ['Fareham', 'contains', 'Portchester West'],
 ['Fareham', 'contains', 'Sarisbury'],
 ['Fareham', 'contains', 'Stubbington'],
 ['Fareham', 'contains', 'Titchfield'],
 ['Fareham', 'contains', 'Titchfield Common'],
 ['Fareham', 'contains', 'Warsash'],
 ['Fareham', 'near_0.19', 'Portsmouth'],
 ['Fareham', 'near_0.29', 'Southampton'],
 ['Fareham', 'near_0.23', 'Eastleigh'],
 ['Fareham', 'near_0.1', 'Gosport'],
 ['Fareham', 'near_0.23', 'Havant'],
 ['Fareham', 'near_0.27', 'Winchester']]

In [459]:
list1 = lads[287]['bounds'][0]+lads[287]['bounds'][1]
list2 = lads[299]['bounds'][0]+lads[299]['bounds'][1]
round(sum([list1[i]-list2[i] for i in [0,1,2,3]]), 2)

-0.11

In [210]:
lads[287]['bounds']

[[-0.21351, 51.51266], [-0.10532, 51.57296]]

In [198]:
lads[299]['bounds']

[[-0.14251, 51.51855], [-0.07633, 51.5755]]

In [20]:
for lad in lads:
    with open('/Users/theojolliffe/Documents/census-data-transformed/json/place/'+lad['code']+'.json', 'w') as outfile:
        json.dump(lad, outfile)

In [21]:
!git add .
!git commit -m "Add files"
!git push -u origin main

[main 647289c] Add files
 337 files changed, 761 insertions(+), 347 deletions(-)
 rewrite json/place/E06000001.json (66%)
 rewrite json/place/E06000002.json (63%)
 rewrite json/place/E06000003.json (67%)
 rewrite json/place/E06000004.json (63%)
 rewrite json/place/E06000005.json (66%)
 rewrite json/place/E06000006.json (67%)
 rewrite json/place/E06000007.json (65%)
 rewrite json/place/E06000008.json (64%)
 rewrite json/place/E06000009.json (66%)
 rewrite json/place/E06000010.json (64%)
 rewrite json/place/E06000011.json (68%)
 rewrite json/place/E06000012.json (66%)
 rewrite json/place/E06000013.json (66%)
 rewrite json/place/E06000014.json (64%)
 rewrite json/place/E06000015.json (66%)
 rewrite json/place/E06000016.json (65%)
 rewrite json/place/E06000017.json (67%)
 rewrite json/place/E06000018.json (65%)
 rewrite json/place/E06000019.json (66%)
 rewrite json/place/E06000020.json (64%)
 rewrite json/place/E06000021.json (65%)
 rewrite json/place/E06000022.json (65%)
 rewrite json/pla

In [ ]:
for ward in wards:
    with open('/Users/theojolliffe/Documents/census-data-ranked/json/place/'+ward['code']+'.json', 'w') as outfile:
        json.dump(ward, outfile)

In [ ]:
for region in regions:
    with open('/Users/theojolliffe/Documents/census-data-ranked/json/place/'+region['code']+'.json', 'w') as outfile:
        json.dump(region, outfile)

In [ ]:
for country in countries:
    with open('/Users/theojolliffe/Documents/census-data-ranked/json/place/'+country['code']+'.json', 'w') as outfile:
        json.dump(country, outfile)